In [1]:
from graph_tool.all import *
from tqdm import tqdm
import csv

In [6]:
with open('../data/REMAnnotationModelScore_1.csv') as file:
    reader = csv.reader(file)
    counter = 0
    for row in tqdm(reader):
        print(row)
        counter += 1
        if(counter == 50):
            break

49it [00:00, 34640.30it/s]

['chr', 'start', 'end', 'geneID', 'REMID', 'regressionCoefficient', 'pValue', 'normModelScore', 'meanDNase1Signal', 'sdDNase1Signal', 'consortium', 'version']
['chr1', '8000', '9999', 'ENSG00000278267', 'REM0000001', '-0.256695911008484', '0.00117467409165201', '0.5224341191833749', '0.02381156626506024', '0.20529241835168552', 'R', '1']
['chr1', '10000', '11999', 'ENSG00000223972', 'REM0000002', '0.282339597095816', '0.00292529340664151', '1.0', '8.482383975903614', '3.3935585369746892', 'R', '1']
['chr1', '16000', '17999', 'ENSG00000223972', 'REM0000003', '0.153983772651832', '0.10480646042063', '0.3866130429076009', '0.7637601445783132', '0.3955437009964319', 'R', '1']
['chr1', '16000', '17999', 'ENSG00000237613', 'REM0000004', '0.71982319723698', '0.000525639660848767', '1.0', '0.7637601626506024', '0.39554369409406526', 'B', '1']
['chr1', '18000', '19999', 'ENSG00000237613', 'REM0000005', '-0.576610369379893', '0.00365802723596124', '0.7430684116746733', '0.4999579939759036', '0.2

In [2]:
# get dict with REMID as key and corresponding CREM as value
# dict with CREM as key and contained REMs as value
crem_dict = dict()
rem_crem_dict = dict()
with open('../data/clusterREMs_1.csv') as file:
    reader = csv.reader(file)
    for row in tqdm(reader):
        if row[0] != "REMID":
            crem_dict[row[0]] = [row[1], row[3], row[4]]
            if rem_crem_dict.get(row[1]) == None:
                rem_crem_dict[row[1]] = [row[0]]
            else:
                rem_crem_dict[row[1]].append(row[0])

1264526it [00:05, 247458.85it/s]


In [13]:
crem_dict.get("REM0000003")[0]

'CREM0000001'

In [3]:
# get dict with GENE name as key and start and end as value
gene_dict = dict()
with open('../data/GeneAnnotation_1.csv') as file:
    reader = csv.reader(file)
#     counter = 0
    for row in tqdm(reader):
        if row[0] != "chr":
            gene_dict[row[3]] = [int(row[1]), int(row[2]), row[0]]
#             print(row)
#             counter += 1
#             if(counter == 10):
#                 break
len(gene_dict)

58175it [00:00, 467508.88it/s]


58174

In [11]:
g = Graph()
name = g.new_vp("string")
g.vp.name = name
type = g.new_vp("string")
g.vp.type = type
rem = g.new_ep("string")
g.ep.rem = rem
chr = g.new_vp("string")
g.vp.chr = chr
start = g.new_vp("int")
g.vp.start = start
end = g.new_vp("int")
g.vp.end = end
score = g.new_ep("double")
g.edge_properties["score"] = score
p_value = g.new_ep("double")
g.ep.p_value = p_value

done_proteins_dict = {}
done_crems = {}
with open('../data/REMAnnotationModelScore_1.csv') as file:
    reader = csv.reader(file)
#     counter = 0
    for row in tqdm(reader):
        rem_name = row[4]
        gen = row[3]
        crem = crem_dict.get(rem_name)
        if crem != None:
            crem = crem[0]
        if row[1] != "start":
            if crem in done_crems:
                v1 = g.vertex(done_crems.get(crem))
            elif crem != None:
                v1 = g.add_vertex()
                g.vp.name[v1] = crem
                g.vp.type[v1] = "REM"
                g.vp.chr[v1] = row[0]
                g.vp.start[v1] = crem_dict.get(rem_name)[1]
                g.vp.end[v1] = crem_dict.get(rem_name)[2]
                done_crems[crem] = int(v1)
            else:
                v1 = g.add_vertex()
                g.vp.name[v1] = rem_name
                g.vp.type[v1] = "REM"
                g.vp.chr[v1] = row[0]
                g.vp.start[v1] = int(row[1])
                g.vp.end[v1] = int(row[2])
            
            if done_proteins_dict.get(gen) == None:
                v2 = g.add_vertex()
                g.vp.name[v2] = gen
                g.vp.type[v2] = "Gene"
                g.vp.chr[v2] = gene_dict.get(gen)[2]
                g.vp.start[v2] = gene_dict.get(gen)[0]
                g.vp.end[v2] = gene_dict.get(gen)[1]
                done_proteins_dict[gen] = int(v2)
            else:
                v2 = g.vertex(done_proteins_dict.get(gen))
                
            e = g.add_edge(v1, v2)
            g.ep.score[e] = row[7]
            g.ep.p_value[e] = row[6]
            g.ep.rem[e] = rem_name
        
#         counter += 1
#         if(counter == 4):
#             break

2404862it [03:30, 11438.34it/s]


In [12]:
g.save("../data/graph-tool/allCelltypeGraph.gt")

In [ ]:
g = load_graph("../data/graph-tool/CTID_0000006.gt")

In [8]:
g.list_properties()

chr            (vertex)  (type: string)
end            (vertex)  (type: int32_t)
name           (vertex)  (type: string)
start          (vertex)  (type: int32_t)
type           (vertex)  (type: string)
p_value        (edge)    (type: double)
rem            (edge)    (type: string)
score          (edge)    (type: double)


In [ ]:
counter = 0
for e in g.edges():
    counter += 1
    print(g.ep.score[e])
    print(g.ep.p_value[e])
    if(counter == 12):
        break
# rowCount = sum(1 for e in g.edges())
# print(rowCount)

In [24]:
# counter = 0
number_genes_g = set()
for v in g.vertices():
    if g.vp.name[v].startswith("E"):
        number_genes_g.add(g.vp.name[v])
#         counter += 1
#     if(counter == 12):
#         break
len(number_genes_g)

35379